In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../bar_movement/data/'

In [3]:
currency_pair = 'Nzd_Cad'
rounding = 3 if 'Jpy' in currency_pair else 5
year_range = '2022-2023'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M30_{year_range}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods - 1, adjust=True,
                       min_periods=periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))

    return rsi

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def n_macd(short_len=12, long_len=21, signal_period=9, lookback=50):
    sh, lon = pd.Series.ewm(df['Mid_Close'], span=short_len).mean(), pd.Series.ewm(df['Mid_Close'], span=long_len).mean()
    ratio = np.minimum(sh, lon) / np.maximum(sh, lon)
    macd = pd.Series(np.where(sh > lon, 2 - ratio, ratio) - 1)
    n_macd = (macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min() + 0.000001) * 2 - 1

    weights = np.arange(1, signal_period + 1)
    n_macdsignal = n_macd.rolling(window=signal_period).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

    return n_macd, n_macdsignal

def smma(closes, length):
    smma = []

    for i in range(len(closes)):
        if i < length:
            smma.append(closes.iloc[:i + 1,].rolling(length).mean().iloc[-1,])

        else:
            smma.append((smma[i - 1] * (length - 1) + closes[i]) / length)

    return pd.Series(smma)

def impulse_macd(high, low, close, ma_len=34, signal_period=9):    
    def _calc_zlema(series, length):
        ema1 = pd.Series.ewm(series, span=length).mean()
        ema2 = pd.Series.ewm(ema1, span=length).mean()
        diff = ema1 - ema2

        return ema1 + diff
    
    hlc3 = (high + low + close) / 3
    hi = smma(high, ma_len)
    lo = smma(low, ma_len)
    mi = _calc_zlema(hlc3, ma_len)
    md = np.where(mi > hi, mi - hi, np.where(mi < lo, mi - lo, 0))
    sb = pd.Series(md).rolling(signal_period).mean()

    return md, sb

def fractal(lows, highs, window=20):
    assert len(lows) == len(highs)

    fractal_period = 2 * window + 1

    is_support = lows.rolling(fractal_period, center=True).apply(lambda x: x[window] == min(x), raw=True)
    is_resistance = highs.rolling(fractal_period, center=True).apply(lambda x: x[window] == max(x), raw=True)
    
    is_support_indices = pd.Series(is_support.index[is_support == 1.0])
    is_resistance_indices = pd.Series(is_resistance.index[is_resistance == 1.0])

    support_fractal_vals = lows[is_support_indices].reindex(lows.index).ffill()
    resistance_fractal_vals = highs[is_resistance_indices].reindex(highs.index).ffill()

    return support_fractal_vals, resistance_fractal_vals

In [6]:
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['n_macd'], df['n_macdsignal'] = n_macd()
df['impulse_macd'], df['impulse_macdsignal'] = impulse_macd(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['smma200'] = smma(df['Mid_Close'], 200)

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]
pips_level_rounding = 0 if 'Jpy' in currency_pair else 2

In [8]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(macd_type, macd_threshold, invert, sl_multiplier, within_pips_atr_multiplier, ma_key, ma_must_be_between_curr_and_sl):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    trade = None
    pips_between_levels = 1.00 if 'Jpy' in currency_pair else 0.0100

    macd_key, macdsignal_key = ('n_macd', 'n_macdsignal') if macd_type == 'n_macd' else (('impulse_macd', 'impulse_macdsignal') if macd_type == 'impulse_macd' else ('macd', 'macdsignal'))

    for i in range(2, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_ao, curr_bo, curr_mid_open, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        if trade is None:
            macd2, macdsignal2 = df.loc[df.index[i - 2], [macd_key, macdsignal_key]]
            macd1, macdsignal1, atr, mid_close = df.loc[df.index[i - 1], [macd_key, macdsignal_key, 'atr', 'Mid_Close']]
            ma = df.loc[df.index[i - 1], ma_key] if ma_key is not None else None
            crossed_up = macd2 < macdsignal2 and macd1 > macdsignal1 and max([macd2, macdsignal2, macd1, macdsignal1]) < 0
            crossed_down = macd2 > macdsignal2 and macd1 < macdsignal1 and min([macd2, macdsignal2, macd1, macdsignal1]) > 0
            macd_final_threshold = macd_threshold * atr if macd_type != 'n_macd' else macd_threshold
            macd_large_enough = min([abs(macd1), abs(macdsignal1), abs(macd2), abs(macdsignal2)]) >= macd_final_threshold
            nearest_level = round(mid_close, pips_level_rounding)
            close_to_nearest_level = abs(mid_close - nearest_level) < atr * within_pips_atr_multiplier
            buy_stop_loss = round(nearest_level - pips_between_levels, rounding)
            sell_stop_loss = round(nearest_level + pips_between_levels, rounding)
            buy_ma_condition_met = (buy_stop_loss < ma < nearest_level) if (ma is not None and ma_must_be_between_curr_and_sl) else (True)
            sell_ma_condition_met = (nearest_level < ma < sell_stop_loss) if (ma is not None and ma_must_be_between_curr_and_sl) else (True)
            buy_signal = crossed_up and macd_large_enough and close_to_nearest_level and buy_ma_condition_met
            sell_signal = crossed_down and macd_large_enough and close_to_nearest_level and sell_ma_condition_met

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal
 
            if buy_signal:
                open_price = float(curr_ao)
                sl_pips = abs(open_price - buy_stop_loss) * sl_multiplier
                stop_loss = round(open_price - sl_pips, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = sl_pips

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1

            elif sell_signal:
                open_price = float(curr_bo)
                sl_pips = abs(sell_stop_loss - open_price) * sl_multiplier
                stop_loss = round(open_price + sl_pips, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = sl_pips

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [9]:
macd_types = ['n_macd', 'impulse_macd', 'macd']
macd_thresholds = [0.0, 0.5, 0.75, 0.95]
invert_vals = [True, False]
sl_multipliers = [0.25, 0.5, 1.0]
within_pips_atr_multipliers = [0.5, 1.0, 1.5, 2.0]
ma_keys = ['ema200', 'smma200', None]
ma_must_be_between_curr_and_sl_vals = [True, False]

# Best reward: 1277.2619499999946
# Best macd type: n_macd
# Best macd threshold: 0.0
# Best invert val: False
# Best sl multiplier: 0.25
# Best within pips atr multiplier: 2.0
# Best ma key: smma200
# Best ma must be between curr and sl levels val: False

# macd_types = ['n_macd']
# macd_thresholds = [0.0]
# invert_vals = [False]
# sl_multipliers = [0.25]
# within_pips_atr_multipliers = [2.0]
# ma_keys = ['smma200']
# ma_must_be_between_curr_and_sl_vals = [True, False]

all_combos = []

for macd_type in macd_types:
    for macd_threshold in macd_thresholds:
        for invert in invert_vals:
            for sl_multiplier in sl_multipliers:
                for within_pips_atr_multiplier in within_pips_atr_multipliers:
                    for ma_key in ma_keys:
                        for ma_must_be_between_curr_and_sl_val in ma_must_be_between_curr_and_sl_vals:
                            ma_must_be_between_curr_and_sl = ma_must_be_between_curr_and_sl_val if ma_key is not None else False
                            all_combos.append((macd_type, macd_threshold, invert, sl_multiplier, within_pips_atr_multiplier, ma_key, ma_must_be_between_curr_and_sl))

                            if ma_key is None:
                                break

best_macd_type, best_macd_threshold, best_invert, best_sl_multiplier, best_within_pips_atr_multiplier, best_ma_key, best_ma_must_be_between_curr_and_sl = None, None, None, None, None, None, None
top_n_results, best_rewards, best_reward, runs_finished, worst_reward = 10, [], -np.inf, 0, np.inf

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for macd_type, macd_threshold, invert, sl_multiplier, within_pips_atr_multiplier, ma_key, ma_must_be_between_curr_and_s in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(macd_type, macd_threshold, invert, sl_multiplier, within_pips_atr_multiplier, ma_key, ma_must_be_between_curr_and_s)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'macd_type': macd_type, 'macd_threshold': macd_threshold, 'invert': invert, 'sl_multiplier': sl_multiplier, 'within_pips_atr_multiplier': within_pips_atr_multiplier, 'ma_key': ma_key, 'ma_must_be_between_curr_and_sl': ma_must_be_between_curr_and_sl})

    if total_profit > best_reward:
        best_reward = total_profit
        best_macd_type, best_macd_threshold, best_invert, best_sl_multiplier, best_within_pips_atr_multiplier, best_ma_key, best_ma_must_be_between_curr_and_sl = macd_type, macd_threshold, invert, sl_multiplier, within_pips_atr_multiplier, ma_key, ma_must_be_between_curr_and_sl

    if total_profit < worst_reward:
        worst_reward = total_profit
 
    print('Best reward so far: ' + str(best_reward))
    print('Worst reward so far: ' + str(worst_reward))
    print()


Num runs: 1440

-824.5807500000087 -51.84311999999999 -876.4238700000087
Num buys: 29
Num sells: 21
Num trades: 50
Num wins: 7
Num losses: 33
Win streak: 4
Loss streak: 23
Avg pips risked: 0.002530800000000003
Avg win amount: 76.76538714285691
Min win amount: 41.977319999999615
Max win amount: 125.92490999999929
Avg loss amount: -31.672987441860627
Min loss amount: -43.52557999999993
Max loss amount: 0.0
Remaining runs: 1439
Best reward so far: -876.4238700000087
Worst reward so far: -876.4238700000087

-746.8497200000077 -146.74045999999996 -893.5901800000076
Num buys: 65
Num sells: 59
Num trades: 124
Num wins: 26
Num losses: 66
Win streak: 5
Loss streak: 13
Avg pips risked: 0.0026380645161290317
Avg win amount: 76.58626961538454
Min win amount: 39.40089999999944
Max win amount: 209.3724500000005
Avg loss amount: -27.939721734693933
Min loss amount: -43.52557999999993
Max loss amount: 0.0
Remaining runs: 1438
Best reward so far: -876.4238700000087
Worst reward so far: -893.59018000000

In [10]:
# EUR/USD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1277.2619499999946
Best macd type: n_macd
Best macd threshold: 0.0
Best invert val: False
Best sl multiplier: 0.25
Best within pips atr multiplier: 2.0
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 1001, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1277, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 905, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 880, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_m

In [10]:
# USD/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 290101.2280000031
Best macd type: n_macd
Best macd threshold: 0.0
Best invert val: True
Best sl multiplier: 0.5
Best within pips atr multiplier: 1.5
Best ma key: None
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 269789, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 289153, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 270934, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 290101, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multip

In [10]:
# GBP/USD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1401.1019499999957
Best macd type: macd
Best macd threshold: 0.5
Best invert val: False
Best sl multiplier: 0.5
Best within pips atr multiplier: 2.0
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 1219, 'macd_type': 'impulse_macd', 'macd_threshold': 0.5, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1379, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1331, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 1.5, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 1157, 'macd_type': 'macd', 'macd_threshold': 0.5, 'invert': False, 'sl_multi

In [10]:
# AUD/USD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 982.8531500000038
Best macd type: macd
Best macd threshold: 0.75
Best invert val: False
Best sl multiplier: 0.5
Best within pips atr multiplier: 1.5
Best ma key: None
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 861, 'macd_type': 'macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 862, 'macd_type': 'macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 817, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 812, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 

In [10]:
# GBP/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 411466.3264999986
Best macd type: n_macd
Best macd threshold: 0.5
Best invert val: True
Best sl multiplier: 0.5
Best within pips atr multiplier: 1.0
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 306528, 'macd_type': 'n_macd', 'macd_threshold': 0.5, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 366883, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 411344, 'macd_type': 'n_macd', 'macd_threshold': 0.5, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 411466, 'macd_type': 'n_macd', 'macd_threshold': 0.5, 'invert': True, 'sl_multiplier': 0.

In [10]:
# EUR/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 239608.89950000058
Best macd type: macd
Best macd threshold: 0.0
Best invert val: False
Best sl multiplier: 0.25
Best within pips atr multiplier: 0.5
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 213151, 'macd_type': 'impulse_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 239505, 'macd_type': 'macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 0.5, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 205161, 'macd_type': 'macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 239608, 'macd_type': 'macd', 'macd_threshold': 0.0, 'invert': False, 'sl_

In [10]:
# GBP/CAD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 2390.186879999972
Best macd type: n_macd
Best macd threshold: 0.0
Best invert val: True
Best sl multiplier: 0.25
Best within pips atr multiplier: 2.0
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 1981, 'macd_type': 'impulse_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 2390, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1983, 'macd_type': 'impulse_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 2363, 'macd_type': 'macd', 'macd_threshold': 0.0, 'invert': False, 'sl

In [10]:
# CAD/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 174057.74350000036
Best macd type: n_macd
Best macd threshold: 0.75
Best invert val: False
Best sl multiplier: 0.25
Best within pips atr multiplier: 0.5
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 171030, 'macd_type': 'n_macd', 'macd_threshold': 0.5, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 151305, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 1.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 173997, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 174057, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False

In [10]:
# GBP/CHF

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1075.731749999998
Best macd type: n_macd
Best macd threshold: 0.95
Best invert val: False
Best sl multiplier: 0.5
Best within pips atr multiplier: 0.5
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 768, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 742, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 770, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 1071, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multipli

In [10]:
# USD/CHF

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1245.741300000004
Best macd type: macd
Best macd threshold: 0.0
Best invert val: True
Best sl multiplier: 0.5
Best within pips atr multiplier: 2.0
Best ma key: None
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 1144, 'macd_type': 'macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1245, 'macd_type': 'macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 1151, 'macd_type': 'macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 1040, 'macd_type': 'macd', 'macd_threshold': 0.95, 'invert': False, 'sl_multiplier': 0.5, 'within_

In [10]:
# NZD/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 94200.93000000046
Best macd type: n_macd
Best macd threshold: 0.75
Best invert val: True
Best sl multiplier: 0.5
Best within pips atr multiplier: 1.5
Best ma key: None
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 90875, 'macd_type': 'macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 89706, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 90723, 'macd_type': 'macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 0.5, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 78532, 'macd_type': 'impulse_macd', 'macd_threshold': 0.95, 'invert': False, 'sl_mult

In [10]:
# AUD/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 138034.90449999933
Best macd type: n_macd
Best macd threshold: 0.75
Best invert val: True
Best sl multiplier: 0.5
Best within pips atr multiplier: 2.0
Best ma key: ema200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 101906, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 100512, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 103322, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': True, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 96974, 'macd_type': 'n_macd', 'macd_threshold': 0.95, 'invert': True, 'sl_m

In [10]:
# CHF/JPY

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 139039.7189999993
Best macd type: n_macd
Best macd threshold: 0.75
Best invert val: True
Best sl multiplier: 1.0
Best within pips atr multiplier: 1.5
Best ma key: ema200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 133602, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 133679, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': True, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 137818, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': True, 'sl_multiplier': 1.0, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 133703, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': True, 

In [10]:
# EUR/CAD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1319.1586100000036
Best macd type: impulse_macd
Best macd threshold: 0.0
Best invert val: True
Best sl multiplier: 0.25
Best within pips atr multiplier: 2.0
Best ma key: ema200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 961, 'macd_type': 'macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1032, 'macd_type': 'n_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 0.5, 'ma_key': 'ema200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 914, 'macd_type': 'n_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.25, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 927, 'macd_type': 'macd', 'macd_threshold': 0.95, 'invert': False, 'sl_m

In [10]:
# EUR/AUD

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best macd type: ' + str(best_macd_type))
print('Best macd threshold: ' + str(best_macd_threshold))
print('Best invert val: ' + str(best_invert))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best within pips atr multiplier: ' + str(best_within_pips_atr_multiplier))
print('Best ma key: ' + str(best_ma_key))
print('Best ma must be between curr and sl levels val: ' + str(best_ma_must_be_between_curr_and_sl))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 2828.9956199999942
Best macd type: impulse_macd
Best macd threshold: 0.0
Best invert val: False
Best sl multiplier: 0.5
Best within pips atr multiplier: 1.5
Best ma key: smma200
Best ma must be between curr and sl levels val: False
-----------------------
Top results:
{'reward': 1915, 'macd_type': 'impulse_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 2165, 'macd_type': 'impulse_macd', 'macd_threshold': 0.0, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 2.0, 'ma_key': 'smma200', 'ma_must_be_between_curr_and_sl': False}
{'reward': 1916, 'macd_type': 'impulse_macd', 'macd_threshold': 0.75, 'invert': False, 'sl_multiplier': 0.5, 'within_pips_atr_multiplier': 1.5, 'ma_key': None, 'ma_must_be_between_curr_and_sl': False}
{'reward': 2828, 'macd_type': 'impulse_macd', 'macd_threshold': 0.